# Alpaca-py crypto trading basic

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/alpacahq/alpaca-py/blob/master/examples/crypto-trading-basic.ipynb)

- This notebook shows how to use alpaca-py with crypto trading API endpoints
- Please use ``paper account``. Please ``DO NOT`` use this notebook with live account. In this notebook, we place orders for crypto as an example.

In [1]:
# Please change the following to your own PAPER api key and secret
# or set them as environment variables (ALPACA_API_KEY, ALPACA_SECRET_KEY).
# You can get them from https://alpaca.markets/
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)
api_key = None
secret_key = None
TRADE_API_KEY=api_key
TRADE_API_SECRET=secret_key

#### We use paper environment for this example ####
paper = True # Please do not modify this. This example is for paper trading only.
####

# Below are the variables for development this documents
# Please do not change these variables
trade_api_url = None
trade_api_wss = None
data_api_url = None
stream_data_wss = None

In [2]:
import os

if api_key is None:
    api_key = os.environ.get('ALPACA_API_KEY')

if secret_key is None:
    secret_key = os.environ.get('ALPACA_SECRET_KEY')

In [3]:
# install alpaca-py if it is not available
try:
    import alpaca
except ImportError:
    !python3 -m pip install alpaca-py
    import alpaca

In [4]:
import json
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

from alpaca.trading.client import TradingClient
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.historical.crypto import CryptoHistoricalDataClient
from alpaca.trading.stream import TradingStream
from alpaca.data.live.crypto import CryptoDataStream

from alpaca.data.requests import (
    CryptoBarsRequest,
    CryptoQuoteRequest,
    CryptoTradesRequest,
    CryptoLatestQuoteRequest
    )
from alpaca.trading.requests import (
    GetAssetsRequest,
    MarketOrderRequest,
    LimitOrderRequest,
    StopLimitOrderRequest,
    GetOrdersRequest,
    ClosePositionRequest
)
from alpaca.trading.enums import (
    AssetClass,
    AssetStatus,
    OrderSide,
    OrderType,
    TimeInForce,
    QueryOrderStatus
)
from alpaca.common.exceptions import APIError

In [5]:
# to run async code in jupyter notebook
import nest_asyncio
nest_asyncio.apply()

In [6]:
# check version of alpaca-py
alpaca.__version__

'0.37.0'

# Trading Client

In [7]:
# setup clients
trade_client = TradingClient(api_key=api_key, secret_key=secret_key, paper=paper, url_override=trade_api_url)

In [8]:
# check trading account
# You can check definition of each field in the following documents
# ref. https://docs.alpaca.markets/docs/account-plans
# ref. https://docs.alpaca.markets/reference/getaccount-1
acct = trade_client.get_account()
acct

{   'account_blocked': False,
    'account_number': 'PA35K0EPAZFK',
    'accrued_fees': '0',
    'buying_power': '181748.91',
    'cash': '99563.76',
    'created_at': datetime.datetime(2025, 2, 1, 10, 53, 4, 229983, tzinfo=TzInfo(UTC)),
    'crypto_status': <AccountStatus.ACTIVE: 'ACTIVE'>,
    'currency': 'USD',
    'daytrade_count': 15,
    'daytrading_buying_power': '0',
    'equity': '99563.76',
    'id': UUID('c096308c-0799-4c30-8a04-fd8003ee16e2'),
    'initial_margin': '8689.31',
    'last_equity': '100000',
    'last_maintenance_margin': '0',
    'long_market_value': '0',
    'maintenance_margin': '0',
    'multiplier': '2',
    'non_marginable_buying_power': '90874.45',
    'options_approved_level': 3,
    'options_buying_power': '82185.15',
    'options_trading_level': 3,
    'pattern_day_trader': True,
    'pending_transfer_in': None,
    'pending_transfer_out': None,
    'portfolio_value': '99563.76',
    'regt_buying_power': '181748.91',
    'short_market_value': '0',
   

In [9]:
# check account configuration
# ref. https://docs.alpaca.markets/reference/getaccountconfig-1
acct_config = trade_client.get_account_configurations()
acct_config

{   'dtbp_check': <DTBPCheck.ENTRY: 'entry'>,
    'fractional_trading': True,
    'max_margin_multiplier': '4',
    'max_options_trading_level': None,
    'no_shorting': False,
    'pdt_check': <PDTCheck.ENTRY: 'entry'>,
    'ptp_no_exception_entry': False,
    'suspend_trade': False,
    'trade_confirm_email': <TradeConfirmationEmail.ALL: 'all'>}

In [10]:
# get list of crypto pairs
# ref. https://docs.alpaca.markets/reference/get-v2-assets-1
req = GetAssetsRequest(
  asset_class=AssetClass.CRYPTO,
  status=AssetStatus.ACTIVE
)
assets = trade_client.get_all_assets(req)
assets[:2]

[{   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
     'attributes': [],
     'easy_to_borrow': False,
     'exchange': <AssetExchange.CRYPTO: 'CRYPTO'>,
     'fractionable': True,
     'id': UUID('dfb6d1da-427c-42b6-864d-f227af29e2e9'),
     'maintenance_margin_requirement': 100.0,
     'marginable': False,
     'min_order_size': 0.002719075,
     'min_trade_increment': 1e-09,
     'name': 'Aave / USD Tether',
     'price_increment': 0.01,
     'shortable': False,
     'status': <AssetStatus.ACTIVE: 'active'>,
     'symbol': 'AAVE/USDT',
     'tradable': True},
 {   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
     'attributes': [],
     'easy_to_borrow': False,
     'exchange': <AssetExchange.CRYPTO: 'CRYPTO'>,
     'fractionable': True,
     'id': UUID('0515189d-1933-4a94-89ce-4e9a24356d58'),
     'maintenance_margin_requirement': 100.0,
     'marginable': False,
     'min_order_size': 0.035984166,
     'min_trade_increment': 1e-09,
     'name': 'Avalanche / US Dollar',
     'pric

### Orders

In [11]:
# we will place orders which Alapca trading platform supports
# - order types for crypto: market, limit, stop_limit
# - time_in_force for crypto: gtc, ioc.
# please refer to the following documents for more details
# ref. https://docs.alpaca.markets/docs/orders-at-alpaca
# ref. https://docs.alpaca.markets/reference/postorder


# we will place orders for symbol: BTC/USD in this example
symbol = "BTC/USD"

In [12]:
# simple, market order
# you can specify:
# or notional value (e.g. 100 USD) (which is in the next example)
req = MarketOrderRequest(
    symbol = symbol,
    qty = 0.01,
    side = OrderSide.BUY,
    type = OrderType.MARKET,
    time_in_force = TimeInForce.GTC,
)
res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
    'asset_id': UUID('276e2673-764b-4ab6-a611-caf665ca6340'),
    'canceled_at': None,
    'client_order_id': '9c3adf8a-6884-453f-a0d6-fa1f7f88f180',
    'created_at': datetime.datetime(2025, 2, 4, 6, 3, 9, 231410, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('75499ef0-2612-4471-a9c2-e1c3e8d6fc8b'),
    'legs': None,
    'limit_price': None,
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.MARKET: 'market'>,
    'position_intent': <PositionIntent.BUY_TO_OPEN: 'buy_to_open'>,
    'qty': '0.01',
    'ratio_qty': None,
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderStatus.PENDING_NEW: 'pending_new'>,
    'stop_price': None,
    'submitted_at': datetime

In [13]:
# simple, market order, notional

req = MarketOrderRequest(
    symbol = symbol,
    notional = 11.1,  # notional is specified in USD, here we specify $1.11
    side = OrderSide.BUY,
    type = OrderType.MARKET,
    time_in_force = TimeInForce.GTC,
)
res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
    'asset_id': UUID('276e2673-764b-4ab6-a611-caf665ca6340'),
    'canceled_at': None,
    'client_order_id': '077744a0-e702-4cc6-98a8-86490878104c',
    'created_at': datetime.datetime(2025, 2, 4, 6, 3, 9, 512133, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('e7fdf75f-ea2d-48b4-aa58-ec1b8442d8a5'),
    'legs': None,
    'limit_price': None,
    'notional': '11.1',
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.MARKET: 'market'>,
    'position_intent': <PositionIntent.BUY_TO_OPEN: 'buy_to_open'>,
    'qty': None,
    'ratio_qty': None,
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderStatus.PENDING_NEW: 'pending_new'>,
    'stop_price': None,
    'submitted_at': datetime

In [14]:
# simple, limit order
req = LimitOrderRequest(
    symbol = symbol,
    qty = 0.01,
    limit_price = 60000,
    side = OrderSide.BUY,
    type = OrderType.LIMIT,
    time_in_force = TimeInForce.GTC,
)
res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
    'asset_id': UUID('276e2673-764b-4ab6-a611-caf665ca6340'),
    'canceled_at': None,
    'client_order_id': '8f4c44f2-b496-45ef-93f1-957816809267',
    'created_at': datetime.datetime(2025, 2, 4, 6, 3, 9, 786619, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('aee49d70-30bf-4a07-bcf1-db45efca3620'),
    'legs': None,
    'limit_price': '60000',
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.LIMIT: 'limit'>,
    'position_intent': <PositionIntent.BUY_TO_OPEN: 'buy_to_open'>,
    'qty': '0.01',
    'ratio_qty': None,
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderStatus.PENDING_NEW: 'pending_new'>,
    'stop_price': None,
    'submitted_at': datetim

In [15]:
# stop limit order
req = StopLimitOrderRequest(
                    symbol = symbol,
                    qty = 0.01,
                    side = OrderSide.BUY,
                    time_in_force = TimeInForce.GTC,
                    limit_price = 55000,
                    stop_price = 60000
                    )

res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
    'asset_id': UUID('276e2673-764b-4ab6-a611-caf665ca6340'),
    'canceled_at': None,
    'client_order_id': '8e4220c0-78a4-4bde-936e-ec73cb79d514',
    'created_at': datetime.datetime(2025, 2, 4, 6, 3, 10, 69053, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('e4c5d120-4326-40d6-a799-a4551b323571'),
    'legs': None,
    'limit_price': '55000',
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.STOP_LIMIT: 'stop_limit'>,
    'position_intent': <PositionIntent.BUY_TO_OPEN: 'buy_to_open'>,
    'qty': '0.01',
    'ratio_qty': None,
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderStatus.NEW: 'new'>,
    'stop_price': '60000',
    'submitted_at': datetime.d

In [16]:
# get a list of orders including closed (e.g. filled) orders by specifying symbol
req = GetOrdersRequest(
    status = QueryOrderStatus.ALL,
    symbols = [symbol]
)
orders = trade_client.get_orders(req)
orders

[{   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
     'asset_id': UUID('276e2673-764b-4ab6-a611-caf665ca6340'),
     'canceled_at': None,
     'client_order_id': '8e4220c0-78a4-4bde-936e-ec73cb79d514',
     'created_at': datetime.datetime(2025, 2, 4, 6, 3, 10, 69053, tzinfo=TzInfo(UTC)),
     'expired_at': None,
     'extended_hours': False,
     'failed_at': None,
     'filled_at': None,
     'filled_avg_price': None,
     'filled_qty': '0',
     'hwm': None,
     'id': UUID('e4c5d120-4326-40d6-a799-a4551b323571'),
     'legs': None,
     'limit_price': '55000',
     'notional': None,
     'order_class': <OrderClass.SIMPLE: 'simple'>,
     'order_type': <OrderType.STOP_LIMIT: 'stop_limit'>,
     'position_intent': <PositionIntent.BUY_TO_OPEN: 'buy_to_open'>,
     'qty': '0.01',
     'ratio_qty': None,
     'replaced_at': None,
     'replaced_by': None,
     'replaces': None,
     'side': <OrderSide.BUY: 'buy'>,
     'status': <OrderStatus.NEW: 'new'>,
     'stop_price': '60000',
   

In [17]:
# see all open orders
req = GetOrdersRequest(
    status = QueryOrderStatus.OPEN,
    symbols = [symbol]
)
open_orders = trade_client.get_orders(req)
open_orders

[{   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
     'asset_id': UUID('276e2673-764b-4ab6-a611-caf665ca6340'),
     'canceled_at': None,
     'client_order_id': '8e4220c0-78a4-4bde-936e-ec73cb79d514',
     'created_at': datetime.datetime(2025, 2, 4, 6, 3, 10, 69053, tzinfo=TzInfo(UTC)),
     'expired_at': None,
     'extended_hours': False,
     'failed_at': None,
     'filled_at': None,
     'filled_avg_price': None,
     'filled_qty': '0',
     'hwm': None,
     'id': UUID('e4c5d120-4326-40d6-a799-a4551b323571'),
     'legs': None,
     'limit_price': '55000',
     'notional': None,
     'order_class': <OrderClass.SIMPLE: 'simple'>,
     'order_type': <OrderType.STOP_LIMIT: 'stop_limit'>,
     'position_intent': <PositionIntent.BUY_TO_OPEN: 'buy_to_open'>,
     'qty': '0.01',
     'ratio_qty': None,
     'replaced_at': None,
     'replaced_by': None,
     'replaces': None,
     'side': <OrderSide.BUY: 'buy'>,
     'status': <OrderStatus.NEW: 'new'>,
     'stop_price': '60000',
   

In [18]:
# cancel all open orders
trade_client.cancel_orders()

[{   'body': None,
     'id': UUID('aee49d70-30bf-4a07-bcf1-db45efca3620'),
     'status': 200},
 {   'body': None,
     'id': UUID('c4ca76fb-bfcc-4009-a315-449d64862b5d'),
     'status': 200},
 {   'body': None,
     'id': UUID('e4c5d120-4326-40d6-a799-a4551b323571'),
     'status': 200}]

### Positions

In [19]:
# '/' not being supported for position calls
symbol="BTCUSD"

In [20]:
# get all open positions
# ref. https://docs.alpaca.markets/reference/getallopenpositions-1
positions = trade_client.get_all_positions()
positions

[{   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
     'asset_id': UUID('64bbff51-59d6-4b3c-9351-13ad85e3c752'),
     'asset_marginable': False,
     'avg_entry_price': '99321.929290289',
     'avg_entry_swap_rate': None,
     'change_today': '-0.0178425180310845',
     'cost_basis': '1001.589053',
     'current_price': '99332.2',
     'exchange': <AssetExchange.CRYPTO: 'CRYPTO'>,
     'lastday_price': '101136.734',
     'market_value': '1001.692625',
     'qty': '0.010084269',
     'qty_available': '0.010084269',
     'side': <PositionSide.LONG: 'long'>,
     'swap_rate': None,
     'symbol': 'BTCUSD',
     'unrealized_intraday_pl': '0.103572',
     'unrealized_intraday_plpc': '0.0001034076797163',
     'unrealized_pl': '0.103572',
     'unrealized_plpc': '0.0001034076797163',
     'usd': None}]

In [21]:
# get positions by symbol
# ref. https://docs.alpaca.markets/reference/getopenposition-1
position = trade_client.get_open_position(symbol_or_asset_id=symbol)
position


{   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
    'asset_id': UUID('64bbff51-59d6-4b3c-9351-13ad85e3c752'),
    'asset_marginable': False,
    'avg_entry_price': '99321.929290289',
    'avg_entry_swap_rate': None,
    'change_today': '-0.0178425180310845',
    'cost_basis': '1001.589053',
    'current_price': '99332.2',
    'exchange': <AssetExchange.CRYPTO: 'CRYPTO'>,
    'lastday_price': '101136.734',
    'market_value': '1001.692625',
    'qty': '0.010084269',
    'qty_available': '0.010084269',
    'side': <PositionSide.LONG: 'long'>,
    'swap_rate': None,
    'symbol': 'BTCUSD',
    'unrealized_intraday_pl': '0.103572',
    'unrealized_intraday_plpc': '0.0001034076797163',
    'unrealized_pl': '0.103572',
    'unrealized_plpc': '0.0001034076797163',
    'usd': None}

In [22]:
# get positions by asset_id
trade_client.get_open_position(symbol_or_asset_id=position.asset_id)

{   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
    'asset_id': UUID('64bbff51-59d6-4b3c-9351-13ad85e3c752'),
    'asset_marginable': False,
    'avg_entry_price': '99321.929290289',
    'avg_entry_swap_rate': None,
    'change_today': '-0.0178425180310845',
    'cost_basis': '1001.589053',
    'current_price': '99332.2',
    'exchange': <AssetExchange.CRYPTO: 'CRYPTO'>,
    'lastday_price': '101136.734',
    'market_value': '1001.692625',
    'qty': '0.010084269',
    'qty_available': '0.010084269',
    'side': <PositionSide.LONG: 'long'>,
    'swap_rate': None,
    'symbol': 'BTCUSD',
    'unrealized_intraday_pl': '0.103572',
    'unrealized_intraday_plpc': '0.0001034076797163',
    'unrealized_pl': '0.103572',
    'unrealized_plpc': '0.0001034076797163',
    'usd': None}

In [23]:
# close the position with specifying qty
# ref. https://docs.alpaca.markets/reference/deleteopenposition-1
trade_client.close_position(
    symbol_or_asset_id = symbol,
    close_options = ClosePositionRequest(
        qty = "0.01",
    )
)

{   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
    'asset_id': UUID('276e2673-764b-4ab6-a611-caf665ca6340'),
    'canceled_at': None,
    'client_order_id': 'eb4e00cd-7224-4020-b853-7973efc6a4fd',
    'created_at': datetime.datetime(2025, 2, 4, 6, 3, 11, 984003, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('7c1d9650-bc3e-46b0-bb1a-fda48d63cf4d'),
    'legs': None,
    'limit_price': None,
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.MARKET: 'market'>,
    'position_intent': <PositionIntent.SELL_TO_CLOSE: 'sell_to_close'>,
    'qty': '0.01',
    'ratio_qty': None,
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.SELL: 'sell'>,
    'status': <OrderStatus.PENDING_NEW: 'pending_new'>,
    'stop_price': None,
    'submitted_at': d

# Trade Update (Stream)

With TradingStream client, you can get updates about trades

You can open this notebook in another window and run below cell to check trade updates.

In [24]:
# subscribe trade updates
trade_stream_client = TradingStream(api_key, secret_key, paper=paper, url_override = trade_api_wss)

async def trade_updates_handler(data):
    print(data)

trade_stream_client.subscribe_trade_updates(trade_updates_handler)
trade_stream_client.run()

event='pending_new' execution_id=None order={   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
    'asset_id': UUID('276e2673-764b-4ab6-a611-caf665ca6340'),
    'canceled_at': None,
    'client_order_id': 'c2b2c161-3f7f-40e0-9662-d87d17b8c25b',
    'created_at': datetime.datetime(2025, 2, 4, 6, 3, 52, 110596, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('656bd876-e62a-4eeb-b4dc-4508553208de'),
    'legs': None,
    'limit_price': None,
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.MARKET: 'market'>,
    'position_intent': <PositionIntent.SELL_TO_CLOSE: 'sell_to_close'>,
    'qty': '0.000084269',
    'ratio_qty': None,
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.SELL: 'sell'>,
    'status': <OrderStatus.PENDING_NEW: 'pending_n

# Market Data (Historical)

In [46]:
# setup crypto historical data client
crypto_historical_data_client = CryptoHistoricalDataClient()

symbol="BTC/USD"

In [ ]:
# get historical bars by symbol
# ref. https://docs.alpaca.markets/reference/cryptobars-1
now = datetime.now(ZoneInfo("America/New_York"))
req = CryptoBarsRequest(
    symbol_or_symbols = [symbol],
    timeframe=TimeFrame(amount = 1, unit = TimeFrameUnit.Hour), # specify timeframe
    start = now - timedelta(days = 1),                          # specify start datetime, default=the beginning of the current day.
    # end_date=None,                                        # specify end datetime, default=now
    limit = 2,                                               # specify limit
)
crypto_historical_data_client.get_crypto_bars(req).df

In [ ]:
# get historical quote by symbol
# ref. https://docs.alpaca.markets/reference/cryptoquotes-1
now = datetime.now(ZoneInfo("America/New_York"))
req = CryptoQuoteRequest(
    symbol_or_symbols = [symbol],
    start = now - timedelta(days = 1),                          # specify start datetime, default=the beginning of the current day.
    # end_date=None,                                        # specify end datetime, default=now
    limit = 2,                                               # specify limit
)
crypto_historical_data_client.get_crypto_quotes(req).df

In [ ]:
# get historical trades by symbol
req = CryptoTradesRequest(
    symbol_or_symbols = [symbol],
    start = now - timedelta(days = 1),                          # specify start datetime, default=the beginning of the current day.
    # end=None,                                             # specify end datetime, default=now
    limit = 2,                                                # specify limit
)
crypto_historical_data_client.get_crypto_trades(req).df

In [ ]:
# get latest quotes by symbol
req = CryptoLatestQuoteRequest(
    symbol_or_symbols = [symbol],
)
res = crypto_historical_data_client.get_crypto_latest_quote(req)
res

: 